# Image

In [1]:
# importing the libraries required.
import numpy as np
import cv2
import timeit

# reading the image
image = cv2.imread("tailstar.jpg")
image1 = image.copy()

# Setting a resolution to resize image for faster processing of Kadanes Algo.
res = 150
image = cv2.resize(image, (res, res),interpolation = cv2.INTER_NEAREST)

# Grayscaling of Image as Basic image will not be a 2d matrix and grayscaling can help us make it a 2d matrix.
gray1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# By default type is int8 which ranges from 0 to 256 so to also have negetive numbers we convert to int 16.
gray1 = gray1.astype(np.int16)

# Using Gaussian blur to smooth the image.
gray1 = cv2.GaussianBlur(gray1,(45,45),0)

print("Gray-scaled array is",gray1)
print("\n Shape is ",gray1.shape)

# Optimizing for beetter results wrt Kadanes Implementaion.
gray1 = gray1 - gray1.max() + 1

print("Final matrix to be passed into Kadanes algo",gray1)

Gray-scaled array is [[14 14 14 ... 16 16 16]
 [14 14 14 ... 16 16 16]
 [14 14 14 ... 16 16 16]
 ...
 [ 8  8  8 ... 12 12 12]
 [ 8  8  8 ... 11 12 12]
 [ 8  8  8 ... 11 12 12]]

 Shape is  (150, 150)
Final matrix to be passed into Kadanes algo [[-49 -49 -49 ... -47 -47 -47]
 [-49 -49 -49 ... -47 -47 -47]
 [-49 -49 -49 ... -47 -47 -47]
 ...
 [-55 -55 -55 ... -51 -51 -51]
 [-55 -55 -55 ... -52 -51 -51]
 [-55 -55 -55 ... -52 -51 -51]]


In [2]:
# Kadanes algorithm
def kadaneAlgo(array, start, finish, n):
    Sum = 0
    maxSum = -99999
    finish[0] = -1
    firstIndex = 0   
    for i in range(n):
        Sum += array[i]
        if Sum < 0:
            Sum = 0
            firstIndex = i + 1
        elif Sum > maxSum:
            maxSum = Sum
            start[0] = firstIndex
            finish[0] = i
    return maxSum

In [3]:
def findMaxSum(M):
    global finalBottom ,finalTop, finalRight , finalLeft , ROW, COL
    maxSum = -9999
    finalLeft = finalRight = finalTop = finalBottom = Left = Right = i = Sum =0
    start = finish = [0]
    for Left in range(COL):
        temp = [0] * ROW
        for Right in range(Left, COL):
            for i in range(ROW):
                temp[i] += M[i][Right]
            Sum = kadaneAlgo(temp, start, finish, ROW)
            if Sum > maxSum:
                maxSum = Sum
                finalLeft = Left
                finalRight = Right
                finalTop = start[0]
                finalBottom = finish[0]
  
    print("(Top, Left)", "(", finalTop,finalLeft, ")")
    print("(Bottom, Right)", "(", finalBottom,finalRight, ")")
    print("Max sum is:", maxSum)

ROW = gray1.shape[0]
COL = gray1.shape[1]
start = timeit.default_timer()
findMaxSum(gray1)
stop = timeit.default_timer()
print('Time: ', stop - start) 

(Top, Left) ( 80 74 )
(Bottom, Right) ( 80 75 )
Max sum is: 2
Time:  1.9949898999999967


In [4]:
TopBottom = int((finalBottom + finalTop)/2)
LeftRight = int((finalLeft + finalRight)/2)
print("Centre of circle is: ",LeftRight,TopBottom)

grayimg = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
LeftRight = int(grayimg.shape[1]/(res/LeftRight))
TopBottom = int(grayimg.shape[0]/(res/TopBottom))
cv2.circle(image1, (LeftRight,TopBottom), 40, (255, 0, 0), 2)
cv2.imshow("Kadane", image1)
cv2.waitKey(0)
cv2.destroyAllWindows()

Centre of circle is:  74 80


# Video

In [1]:
import timeit
LeftRight = TopBottom = 0

def video(gray1,ROWw,COLl):
    
    def kadaneAlgo(array, start, finish, n):
        Sum = 0
        maxSum = -9999
        finish[0] = -1
        local_start = 0
 
        for i in range(n):
            Sum += array[i]
            if Sum < 0:
                Sum = 0
                local_start = i + 1
            elif Sum > maxSum:
                maxSum = Sum
                start[0] = local_start
                finish[0] = i

        if finish[0] != -1:
            return maxSum

        maxSum = array[0]
        start[0] = finish[0] = 0

        for i in range(1, n):
            if array[i] > maxSum:
                maxSum = array[i]
                start[0] = finish[0] = i
        return maxSum

    def findMaxSum(M):
        global Rightrow ,Rightcol, Leftcol , Leftrow ,  ROW, COL,LeftRight,TopBottom
        maxSum  = -9999
        finalLeft = finalRight = finalTop = finalBottom = left = right = i = 0
 
        temp = [None] * ROW
        Sum = 0
        start = [0]
        finish = [0]

        for left in range(COL):
            temp = [0] * ROW
            for right in range(left, COL):
                for i in range(ROW):
                    temp[i] += M[i][right]
                Sum = kadaneAlgo(temp, start, finish, ROW)
                if Sum > maxSum:
                    maxSum = Sum
                    finalLeft = left
                    finalRight = right
                    finalTop = start[0]
                    finalBottom = finish[0]

        Leftrow = finalTop
        Leftcol = finalLeft    
        print("(Top, Left)", "(", finalTop,finalLeft, ")")
        Rightrow = finalRight
        Rightcol = finalBottom
        print("(Bottom, Right)", "(", finalBottom,finalRight, ")")
        print("Max sum is:", maxSum)

        print("leftrow",Leftrow)
        TopBottom = int((Rightcol + Leftrow)/2)
        LeftRight = int((Leftcol + Rightrow)/2)
    
    print("TopBottom",TopBottom)
    print("LeftRight",LeftRight)
    ROW = ROWw
    COL = COLl
    start = timeit.default_timer()
    findMaxSum(gray1)
    stop = timeit.default_timer()
    print('Time: ', stop - start) 
    return LeftRight,TopBottom

In [3]:
#import the necessary packages

import numpy as np
import cv2
import math
cap = cv2.VideoCapture('lantern.mp4')


while(cap.isOpened()):
    ret, image = cap.read()
    if ret == False:
        break
    res = 60
    image1 = cv2.resize(image, (res, res),interpolation = cv2.INTER_NEAREST)
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray1 = gray1.astype(np.int16)
    gray1 = cv2.GaussianBlur(gray1,(45,45),0)
    gray1 = gray1 - gray1.max() + 1
    ROW = gray1.shape[0]
    COL = gray1.shape[1]
    video(gray1,ROW,COL)
    print("TopBottomin",TopBottom)
    print("LeftRightin",LeftRight)
    grayimg = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    LeftRight = math.ceil(grayimg.shape[1]/(res/LeftRight))
    TopBottom = math.ceil(grayimg.shape[0]/(res/TopBottom))
    cv2.circle(image, (LeftRight,TopBottom), 41, (255, 0, 0), 2)
    cv2.imshow("Kadane", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



TopBottom 180
LeftRight 214
(Top, Left) ( 31 33 )
(Bottom, Right) ( 33 34 )
Max sum is: 2
leftrow 31
Time:  0.14876050000000873
TopBottomin 32
LeftRightin 33
TopBottom 326
LeftRight 704
(Top, Left) ( 31 33 )
(Bottom, Right) ( 33 34 )
Max sum is: 2
leftrow 31
Time:  0.17788609999999494
TopBottomin 32
LeftRightin 33
TopBottom 326
LeftRight 704
(Top, Left) ( 31 32 )
(Bottom, Right) ( 33 35 )
Max sum is: 9
leftrow 31
Time:  0.19296460000001048
TopBottomin 32
LeftRightin 33
TopBottom 326
LeftRight 704
(Top, Left) ( 30 33 )
(Bottom, Right) ( 33 35 )
Max sum is: 8
leftrow 30
Time:  0.18362350000001015
TopBottomin 31
LeftRightin 34
TopBottom 316
LeftRight 726
(Top, Left) ( 30 33 )
(Bottom, Right) ( 32 34 )
Max sum is: 2
leftrow 30
Time:  0.24123979999998824
TopBottomin 31
LeftRightin 33
TopBottom 316
LeftRight 704
(Top, Left) ( 30 33 )
(Bottom, Right) ( 33 35 )
Max sum is: 5
leftrow 30
Time:  0.16909429999998338
TopBottomin 31
LeftRightin 34
TopBottom 316
LeftRight 726
(Top, Left) ( 30 33 )
(B

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-6sxsq0tp\opencv\modules\imgproc\src\resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
